# 要約 
以下のノートブックは、Kaggleの「LLM 20 Questions」コンペティションにおけるベースラインモデルの構築を目的としています。主に、特定の言葉を当てるための「20の質問」ゲームへのAIエージェントの参加を支援するもので、特定の言語モデルを利用しています。

### 取り組んでいる問題
ノートブックは、プレイヤーが「はい」または「いいえ」で質問を行い、ターゲットとなる言葉を推測するゲームを支援するために、言語モデルを使用して効果的かつ戦略的な質問と推測を行うためのエージェントを構築しています。このゲームでは、効率よく情報を収集し、限られた質問を通じて答えを導き出す能力が求められます。

### 使用している手法およびライブラリ
1. **モデルの選択と依存関係のインストール**:
   - `bitsandbytes`と`accelerate`、最新の`transformers`ライブラリを使用して、モデルの量子化と効率的なメモリ管理を行います。
   - HuggingFaceから選択した言語モデル（LLAMA3、Phi-3、Qwen-2バリアント）を使用します。

2. **モデルのダウンロードと設定**:
   - HuggingFaceからモデルをダウンロードし、量子化された設定でロードします。これにより、ストレージ使用量を削減し、メモリ効率を最適化します。
   
3. **プロンプト設計**:
   - AI エージェント用のプロンプトを設計し、質問、推測、回答の各モードに応じたプロンプトを構築しています。プロンプトは、モデルにコンテキストを提供し、効果的な質問や推測を行うために必要です。

4. **AIエージェントの実装**:
   - モデルから生成されるレスポンスに基づいて、エージェントが質問する機能 (asker)、推測する機能 (guesser)、および答える機能 (answerer) を持つクラス `Robot` を定義しています。

5. **シミュレーションとテスト**:
   - Kaggle Environmentsを使用して、AIエージェントとダムエージェントとの対戦をシミュレーションし、ゲームの結果やエージェントの性能を確認します。

このノートブックは、言語モデルを用いて「20の質問」ゲームでの対話の流れを形成し、ゲームの戦略を最適化するための基盤を提供しています。

---


# 用語概説 
こちらのJupyter Notebookに関連する専門用語とその簡単な解説を以下に示します。特に初心者が混乱しやすい点や、このノートブック特有の用語にフォーカスしました。

1. **トークナイザー (Tokenizer)**:
   - テキストをトークン（単語やフレーズなどの基本的な単位）に分割するためのツールです。モデルはトークン処理を行うため、自然言語を数値的な表現に変換します。ノートブックにおいては、`tokenizer.apply_chat_template` というメソッドを用いて特別なトークンが適用されています。

2. **量子化 (Quantization)**:
   - モデルのパラメータを低精度（例えば、16ビットや8ビット）で表現することを指します。これにより、メモリ使用量を削減し、計算速度を向上させる目的があります。ノートブックでは、モデルをメモリに量子化してロードすることでストレージを節約しています。

3. **HuggingFace モデルハブ**:
   - HuggingFaceが提供する、さまざまなトランスフォーマーベースのモデルが保存されているオンラインリポジトリです。ユーザーはここからモデルを検索し、ダウンロードして使用することができます。

4. **リモートコードの信頼 (trust_remote_code)**:
   - 外部ソースからのコードを実行する際の安全性を確保するためのパラメータです。信頼できるソースからのコードである場合に設定して、リモートからのモデルや定義を読み込むことを許可します。

5. **メモリ効率の良い分散処理 (Memory-efficient distributed processing)**:
   - 複数のGPUやノードを用いて大規模なデータやモデルを効率良く処理する方法であり、メモリの使用量を抑えながら計算を分散させることを指します。ノートブック内では、この機能が無効にされています。

6. **ターン終了トークン (Turn terminators)**:
   - 会話の一部が終了したことを示すトークンで、モデルが次の反応を生成する際にどの時点で終わったかを認識するために使用されます。ノートブックでは、追加のターン終了トークンを定義して管理しています。

7. **プロンプト (Prompt)**:
   - モデルに対して出力を生成させるための入力テキストや指示のことです。ノートブックでは、「20の質問」ゲームのルールや目的に基づいた指示が与えられ、モデルがその指示に従って応答を生成します。

8. **Kaggle 環境 (Kaggle environment)**:
   - Kaggleのコンペティションやプロジェクトに特化したシミュレーション環境であり、エージェントが対戦するゲーム上での実行環境を指します。これは、Kaggleの`kaggle_environments`ライブラリを使用して構築されます。

これらの解説が、Jupyter Notebookを利用する際の理解を助け、学習に役立つことを願っています。

---


# LLM 20 Questions ベースライン

私は `tokenizer.apply_chat_template` を使用して特別なトークンを自動的に適用したので、モデルやプロンプトを便利に変更できます。


サポートされているモデル:
- `LLAMA3 バリアント`
- `Phi-3 バリアント`
- `Qwen-2 バリアント`

## 前提条件
アクセラレーターを GPU T4 に設定してください。

In [ ]:
%%bash
# submissionという名前のディレクトリを作成します。
mkdir -p /kaggle/working/submission

# modelという名前の一時ディレクトリを作成します。
mkdir -p /tmp/model

# bitsandbytesとaccelerateをインストールします。
pip install -q bitsandbytes accelerate

# transformersを最新版にアップグレードしてインストールします。
pip install -qU transformers

## モデルのダウンロード

### HuggingFace ログイン

1. HuggingFace アクセストークンを発行します（https://huggingface.co/settings/tokens）。

2. HuggingFace アクセストークンをシークレットに追加します。シークレットは `Add-ons -> secrets` で見つけることができます。




![Screenshot 2024-08-01 at 11.40.17 AM.png](attachment:fb5805e5-566e-41f1-ba50-0d9f9fade571.png)

In [ ]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

# HuggingFaceのトークンを保存する変数を初期化します。
HF_TOKEN: str | None  = None

# シークレットからHuggingFaceトークンを取得します。
try:
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
except:
    # トークンの取得に失敗した場合は何もしません。
    pass

### モデルの選択

希望するモデルを [HuggingFace モデルハブ](https://huggingface.co/models) から見つけ、そのモデル名を次のコマンドで使用してください。

サポートされているモデル:
- `LLAMA3 バリアント`
- `Phi-3 バリアント`
- `Qwen-2 バリアント`

In [ ]:
# 使用するモデルのリポジトリIDを設定します。
repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"

### HuggingFace経由でモデルをダウンロード

ディスク使用量を削減するために、モデルを `/tmp/model` にダウンロードします。

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

# モデルの保存先パスを設定します。
g_model_path = Path("/tmp/model")

# 既にモデルのパスが存在する場合は、そのディレクトリを削除します。
if g_model_path.exists():
    shutil.rmtree(g_model_path)

# モデルを保存するための新しいディレクトリを作成します。
g_model_path.mkdir(parents=True)

# HuggingFaceからモデルをダウンロードします。
snapshot_download(
    repo_id=repo_id,  # 使用するモデルのリポジトリID
    ignore_patterns="original*",  # 元のファイルを無視するためのパターン
    local_dir=g_model_path,  # ダウンロード先のローカルディレクトリ
    token=globals().get("HF_TOKEN", None)  # HuggingFaceトークン
)

In [ ]:
# /tmp/model ディレクトリ内のファイルとフォルダをリスト表示します。
!ls -l /tmp/model

### 量子化されたモデルを保存

ダウンロードしたモデルをメモリに量子化してロードします。  
これにより、ストレージを節約できます。

さらに、保存されたモデルはすでに量子化されているため、`main.py`では `bitsandbytes` パッケージを使用する必要はありません。

In [ ]:
# モデルをメモリにロードします。
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# メモリ効率の良い分散処理を無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# ダウンロードされたモデルのパスを設定します。
downloaded_model = "/tmp/model"

# 量子化の設定を行います。
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,  # 4ビットでのロードを有効にします。
    bnb_4bit_compute_dtype=torch.float16,  # 計算時のデータ型を浮動小数点16ビットに設定します。
)

# 量子化されたモデルを事前に読み込みます。
model = AutoModelForCausalLM.from_pretrained(
    downloaded_model,  # 読み込むモデルのパス
    quantization_config = bnb_config,  # 量子化設定
    torch_dtype = torch.float16,  # モデルのデータタイプを浮動小数点16ビットに設定します。
    device_map = "auto",  # デバイスマッピングを自動設定します。
    trust_remote_code = True,  # リモートコードの信頼を有効にします。
)

# 対応するトークナイザーを読み込みます。
tokenizer = AutoTokenizer.from_pretrained(downloaded_model)

In [ ]:
# モデルを提出用ディレクトリに保存します。
model.save_pretrained("/kaggle/working/submission/model")  # モデルを指定のパスに保存します。
tokenizer.save_pretrained("/kaggle/working/submission/model")  # トークナイザーを指定のパスに保存します。

In [ ]:
# メモリからモデルをアンロードします。
import gc, torch

# モデルとトークナイザーを削除します。
del model, tokenizer

# ガーベジコレクションを実行してメモリを解放します。
gc.collect()

# CUDAキャッシュを空にします。
torch.cuda.empty_cache()

## エージェント

### プロンプト

プロンプトは [Anthropic プロンプトライブラリ](https://docs.anthropic.com/en/prompt-library/library) から参照されます。

プロンプトは2つの部分で構成されています：
- `system_prompt`: これはカテゴリを決定するための最初の質問です。
- `chat_history`: これはモデルにコンテキストを提供するためのチャット履歴です。

In [ ]:
%%writefile submission/prompts.py

# プロンプトを管理するための関数を定義します。

def asker_prompt(obs):
    message = []
    
    # システムプロンプト
    ask_prompt = f"""あなたは「20の質問」ゲームをプレイすることに熟練した役立つAIアシスタントです。
ユーザーが考えている言葉を当てるために質問をするのがあなたの役割です。
はい/いいえの質問をすることによって可能性を絞り込んでください。
段階的に考え、最も有益な質問をするようにしてください。
\n"""

    message.append({"role": "system", "content": ask_prompt})

    # チャット履歴の追加
    for q, a in zip(obs.questions, obs.answers):
        message.append({"role": "assistant", "content": q})  # 質問
        message.append({"role": "user", "content": a})  # 回答

    return message


def guesser_prompt(obs):
    message = []
    
    # システムプロンプト
    guess_prompt = f"""あなたは「20の質問」ゲームをプレイすることに熟練した役立つAIアシスタントです。
ユーザーが考えている言葉を当てるのがあなたの役割です。
段階的に考えてください。
\n"""

    # チャット履歴の構築
    chat_history = ""
    for q, a in zip(obs.questions, obs.answers):
        chat_history += f"""質問: {q}\n回答: {a}\n"""
    
    prompt = (
            guess_prompt + f"""これまでのゲームの状態は次の通りです:\n{chat_history}
        この会話に基づいて、言葉を当てることができますか？言葉だけを答えて、冗長な説明は不要です。"""
    )
    
    message.append({"role": "system", "content": prompt})
    
    return message


def answerer_prompt(obs):
    message = []
    
    # システムプロンプト
    prompt = f"""あなたは「20の質問」ゲームをプレイすることに熟練した役立つAIアシスタントです。
ユーザーが考えている言葉を当てるのを助けるために質問に答えるのがあなたの役割です。
あなたの回答は「はい」または「いいえ」でなければなりません。
キーワードは: "{obs.keyword}" です、それはカテゴリ: "{obs.category}" のものです。
ユーザーがキーワードを推測するのを助けるために正確な回答を提供してください。
"""

    message.append({"role": "system", "content": prompt})
    
    # チャット履歴の追加
    message.append({"role": "user", "content": obs.questions[0]})  # 最初の質問
    
    if len(obs.answers) >= 1:
        for q, a in zip(obs.questions[1:], obs.answers):
            message.append({"role": "assistant", "content": a})  # 回答
            message.append({"role": "user", "content": q})  # 次の質問
    
    return message

### エージェント

より多くのLLMモデルを追加するには、ターン終了トークンを終端子リストに追加します。

In [ ]:
%%writefile submission/main.py

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

from prompts import *

# メモリ効率の良い分散処理を無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Kaggleのエージェントパスを設定します。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    MODEL_PATH = os.path.join(KAGGLE_AGENT_PATH, "model")
else:
    MODEL_PATH = "/kaggle/working/submission/model"

# モデルを事前に読み込みます。
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",  # デバイスマッピングを自動設定します。
    trust_remote_code=True,  # リモートコードの信頼を有効にします。
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# 使用するターン終了トークンを指定します。
terminators = [tokenizer.eos_token_id]

# 追加のターン終了トークンの可能性
# llama3, phi3, gwen2 の順で
potential_terminators = ["<|eot_id|>", "<|end|>", "<end_of_turn>"]

# ポテンシャルターン終了トークンをterminatorsリストに追加します。
for token in potential_terminators:
    token_id = tokenizer.convert_tokens_to_ids(token)
    if token_id is not None:
        terminators.append(token_id)

# レスポンスを生成する関数
def generate_response(chat):
    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt").to(model.device)  # 入力テンソルを作成します。
    outputs = model.generate(inputs, max_new_tokens=32, pad_token_id=tokenizer.eos_token_id, eos_token_id=terminators)  # モデルから出力を生成します。
    response = outputs[0][inputs.shape[-1]:]  # 出力の一部を取り出します。
    out = tokenizer.decode(response, skip_special_tokens=True)  # デコードして文字列に戻します。

    return out

# Robotクラスを定義します。
class Robot:
    def __init__(self):
        pass

    def on(self, mode, obs):
        assert mode in [
            "asking", "guessing", "answering",
        ], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            # 質問する役割を実行します。
            output = self.asker(obs)
        if mode == "answering":
            # 回答する役割を実行します。
            output = self.answerer(obs)
            if "yes" in output.lower():
                output = "yes"
            elif "no" in output.lower():
                output = "no"
            if "yes" not in output.lower() and "no" not in output.lower():
                output = "yes"
        if mode == "guessing":
            # 推測する役割を実行します。
            output = self.guesser(obs)
        return output

    def asker(self, obs):
        input = asker_prompt(obs)  # 質問用のプロンプトを取得します。
        output = generate_response(input)  # レスポンスを生成します。        
        return output

    def guesser(self, obs):
        input = guesser_prompt(obs)  # 推測用のプロンプトを取得します。
        output = generate_response(input)  # レスポンスを生成します。
        return output

    def answerer(self, obs):
        input = answerer_prompt(obs)  # 回答用のプロンプトを取得します。
        output = generate_response(input)  # レスポンスを生成します。
        return output

robot = Robot()

# エージェント関数を定義します。
def agent(obs, cfg):

    if obs.turnType == "ask":
        response = robot.on(mode="asking", obs=obs)  # 質問の場合

    elif obs.turnType == "guess":
        response = robot.on(mode="guessing", obs=obs)  # 推測の場合

    elif obs.turnType == "answer":
        response = robot.on(mode="answering", obs=obs)  # 回答の場合

    # レスポンスが空または短すぎる場合には「yes」とする。
    if response == None or len(response) <= 1:
        response = "yes"

    return response  # レスポンスを返します。

## シミュレーション

### pygameのインストール

In [ ]:
# pygameをインストールします。
!pip install pygame

In [ ]:
%%writefile dumb.py

# ダムエージェントを定義します。
def dumb_agent(obs, cfg):
    
    # エージェントが推測者で、ターンタイプが「ask」の場合
    if obs.turnType == "ask":
        response = "それはカモですか？"  # 質問を返します。
    # エージェントが推測者で、ターンタイプが「guess」の場合
    elif obs.turnType == "guess":
        response = "カモ"  # 推測を返します。
    # エージェントが回答者の場合
    elif obs.turnType == "answer":
        response = "いいえ"  # 回答を返します。
    
    return response  # レスポンスを返します。

In [ ]:
%%time

# kaggle_environmentsライブラリを使用して環境を作成します。
from kaggle_environments import make

# "llm_20_questions" 環境をデバッグモードで作成します。
env = make("llm_20_questions", debug=True)

# 2つのメインエージェントと2つのダムエージェントでゲームを実行します。
game_output = env.run(agents=["submission/main.py", "submission/main.py", "dumb.py", "dumb.py"])

In [ ]:
# ゲームの結果をIPython環境で表示します。
env.render(mode="ipython", width=600, height=700)

## エージェントの提出

In [ ]:
# pigzとpvをインストールします。出力は表示しません。
!apt install pigz pv > /dev/null

In [ ]:
# submissionディレクトリを圧縮し、tar.gzファイルを作成します。
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .

In [ ]:
# 追加の操作は行われていません。必要に応じてコードを追加してください。